In [250]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In questo script voglio leggere i documenti e dato che una squadra può avere diversi soprannomi, poi per la classificazione risulta complicato andare a capire di chi si sta parlando. Per questo motivo vado a considerare tutti i soprannomi utilizzati nella lega rispetto alle squadre e in ciascun documento (in questo caso lo faccio su un solo documento) vado a sostituire il nome della squadra e tutti i suoi soprannomi con "home team" o "away team".
I sinonimi vengono presi da un dizionario.

Ho due problemi: 
- ci sono squadre come "Hellas Verona" che può anche essere chiamato "Verona" quindi devo stare attento nel dizionario a posizionare "Hellas Verona" prima di "Verona", altrimenti se posizionassi al contrario, quando nel documento sostituisco con ad esempio "home team", risulta "Hellas home team", invece anche Hellas deve sparire
- un problema simile a quello descritto sopra è per le abbreviazioni come "Juve" e "Juventus" quindi per non trovarci in una situazione simile "home teamntus" dobbiamo mettere la stringa completa prima dell'abbreviata

Un altro problema è legato alla stadio, perché qui il nome di alcuni stadi come "Olimpico di Torino" viene presa la parola Torino e sostituita. Questo dovrebbe succedere o no? Potrei sostituire il nome dello stadio con "home stadium" o "away stadium"

In [251]:
import urllib.request
import json
import spacy
nlp = spacy.load("en_core_web_sm")

tokenize = lambda text: [x.lemma_.lower() for x in text if x.pos_ in ['NOUN', 'PROPN', 'VERB', 'ADJ', 'ADV']]

In [252]:
dataframe = pd.read_csv("description_predictions.csv", index_col=0)

In [253]:
with open("sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [254]:
descriptions = dataframe.iloc[102:] #cambio il numero del record
#documento utile 101 
descriptions = descriptions.iloc[:1]

Estraggo le squadre, perché nel testo ci sono i soprannomi delle squadre e quindi non essendo conosciuti dal vocabolario, li sostituisco con il nome della relativa squadra

In [255]:
h_team, a_team, description = descriptions.h_team.values[0], descriptions.a_team.values[0], descriptions.description.values[0]
print(h_team, a_team, "\n"+description)

syn = {}
for key in jsonObject.keys():
    if (h_team in key) or (key in h_team):
        syn['home team'] = jsonObject[key] 
        
    if (a_team in key) or (key in a_team):
        syn['away team'] = jsonObject[key] 

Atalanta Lazio 
Gewiss Stadium will host a highly-anticipated Serie A game between Atalanta and Lazio. The Orobici came from behind to beat Sampdoria 3-1 in a midweek clash, with the team moving closer to the Champions League berth. Atalanta aim to give fans yet another reason for celebration, and they are likely to stick to their attacking style of play. Slovenia international Josip Ilicic is widely expected to return to the starting XI after finding the net in the midweek triumph, whereas all Pessina, Djimsiti, Gosens, Hateboer, Demiral, and Toloi should miss out with injuries. 

The Biancocelesti, on the other hand, recorded a hard-fought 1-0 win over Fiorentina in their last league match. Lazio, though, are not regarded as good travelers, which is probably the main reason why home win should be considered. Both Adekanye and Zaccagni are sidelined with injuries for the visiting team. 


In [256]:
print("Game: "+h_team+" - "+a_team)
print("ORIGINAL DOCUMENT\n"+description)
description = description.lower()
for key in syn.keys():
    for val in syn[key]:
        description = description.replace(val.lower(), key)

print("\n\nCHANGED DOCUMENT\n"+description)

Game: Atalanta - Lazio
ORIGINAL DOCUMENT
Gewiss Stadium will host a highly-anticipated Serie A game between Atalanta and Lazio. The Orobici came from behind to beat Sampdoria 3-1 in a midweek clash, with the team moving closer to the Champions League berth. Atalanta aim to give fans yet another reason for celebration, and they are likely to stick to their attacking style of play. Slovenia international Josip Ilicic is widely expected to return to the starting XI after finding the net in the midweek triumph, whereas all Pessina, Djimsiti, Gosens, Hateboer, Demiral, and Toloi should miss out with injuries. 

The Biancocelesti, on the other hand, recorded a hard-fought 1-0 win over Fiorentina in their last league match. Lazio, though, are not regarded as good travelers, which is probably the main reason why home win should be considered. Both Adekanye and Zaccagni are sidelined with injuries for the visiting team. 


CHANGED DOCUMENT
gewiss stadium will host a highly-anticipated serie a g